# Face Recognition Model - Serving Function
The function uses sklearn classifier on top of an opencv deep learning model to encode and recognize faces in given image

In [ ]:
#nuclio: ignore
import nuclio

### Install dependencies and set config

In [ ]:
%%nuclio cmd 
pip install cmake
pip install dlib
pip install face_recognition
pip install opencv-contrib-python
pip install imutils
pip install sklearn
pip install pandas
pip install joblib
pip install v3io_frames

In [ ]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

### Perform necessary imports

In [ ]:
import cv2
import face_recognition
import imutils
import joblib
import json
import numpy as np
import pandas as pd
import random
import string
import v3io_frames as v3f
import os
import datetime

### Set function environment variables

In [ ]:
%%nuclio env
MODEL_PATH = /User/demos/face-recognition/artifacts/model.bst
CLASSES_MAP = /User/demos/face-recognition/artifacts/idx2name.csv

In [ ]:
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

### Model serving class

In [ ]:
class SKModel(object):
    def __init__(self):
        self.name = 'model.bst'
        self.model_filepath = os.environ['MODEL_PATH']
        self.model = None
        self.ready = None
        self.classes = os.environ['CLASSES_MAP']
    
    def load(self):
        self.model = joblib.load(self.model_filepath)
        self.ready = True
    
    def predict(self, context, data):
        
        # acquires all metadata 
        time = data['time']
        cam_name = data['camera']
        img_url = data['file_path']
        
        # prepares image for use
        with open(img_url, 'rb') as f:
            content = f.read()
        img_bytes = np.frombuffer(content, dtype=np.uint8)
        image = cv2.imdecode(img_bytes, flags=1)
        
        # converts image format to RGB for comptability with face_recognition library and resize for faster processing
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(image, width=750)
        ratio = image.shape[1] / float(rgb.shape[1])
        
        #gets mapping from label to name and known encodings
        idx2name_df = pd.read_csv(self.classes).set_index('value')
        
        if not self.model:
            self.load()
        
        #locates faces in image and extracts embbeding vector for each face
        context.logger.info_with('[INFO]', msg="recognizing faces...")
        boxes = face_recognition.face_locations(rgb, model='hog')
        encodings = face_recognition.face_encodings(rgb, boxes)
        
        #determines if face is a clear match/ambiguous.
        names = []
        labels = []
        for encoding in encodings:
            name = 'unknown'
            label = 'unknown'
            probs = self.model.predict_proba(encoding.reshape(1, -1))
            if np.max(probs) > 0.5:
                label = np.argmax(probs)
                name = idx2name_df.loc[label]['name'].replace('_', ' ')
            names.append(name)
            labels.append(label)

        #frames client to save necessary data
        client = v3f.Client("framesd:8081", container="users")

        #draw boxes with name on the image and performs logic according to match/ambiguous 
        ret_list = []
        for ((top, right, bottom, left), name, encoding, label) in zip(boxes, names, encodings, labels):  

            #rescale the face coordinates
            top = int(top * ratio)
            right = int(right * ratio)
            bottom = int(bottom * ratio)
            left = int(left * ratio)

            #random string for unique name in our saved data
            rnd_tag = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

            new_row = {}
            #saves all extracted data to kv.  
            new_row = {'c' + str(i).zfill(3): encoding[i] for i in range(128)}
            if (name != 'unknown'): #and (len(enc_df.loc[enc_df['label'] == label]) < 50): TODO add different logic for limit of images of same person in dataset
                new_row['label'] = label
            else:
                new_row['label'] = np.nan
            new_row['imgUrl'] = img_url
            new_row['fileName'] = name.replace(' ', '_') + '_' + rnd_tag 
            new_row['camera'] = cam_name
            new_row['time'] = datetime.datetime.utcnow()
            new_row_df = pd.DataFrame(new_row, index=[0])
            client.write(backend='kv', table='iguazio/demos/face-recognition/artifacts/encodings', dfs=new_row_df, index_cols=['fileName'])

            #appends box and name to the returned list            
            ret_list.append(((top, right, bottom, left), name))

        return ret_list

### Main function
simply initializes the model class and invokes the predict method 

In [ ]:
def handler(context, event):
    model = SKModel()
    return model.predict(context=context, data=event.body)

In [ ]:
#nuclio: end-code

### Set configuration for deployment

In [ ]:
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function
fn = code_to_function('recognize-faces', runtime='nuclio')

# set the API/trigger, attach the home dir to the function
fn.with_http(workers=2).add_volume('User','~/')

# set environment variables
fn.set_env('MODEL_PATH', '/User/demos/face-recognition/artifacts/model.bst')
fn.set_env('CLASSES_MAP', '/User/demos/face-recognition/artifacts/idx2name.csv')
fn.set_env('V3IO_ACCESS_KEY', os.environ['V3IO_ACCESS_KEY'])

### Deploy the function to the cluster
May take a few minutes due to building of image

In [ ]:
addr = fn.deploy(project='default')